Mount Github files to Google Drive

In [730]:
! rm -rf sample_data; curl -s https://raw.githubusercontent.com/DayongTong/MEAM517_Project_Git/main/sync_project.py > sync_project.py
files = ["spacecraft.py","simulation.py", "dynamic_constraints.py", "kinematic_constraints.py"]

from sync_project import sync_project
sync_project(files, False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install Drake

In [731]:
# install pydrake

!curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/colab_drake_setup.py > colab_drake_setup.py
from colab_drake_setup import setup
setup()

<IPython.core.display.Javascript object>

Import Packages

In [732]:
from pydrake.all import eq, MathematicalProgram, Solve, Variable

import matplotlib.pyplot as plt
import numpy as np
import importlib

from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve, SolverType
from pydrake.trajectories import PiecewisePolynomial
from pydrake.solvers.snopt import SnoptSolver


In [733]:
import kinematic_constraints
import dynamic_constraints
importlib.reload(kinematic_constraints)
importlib.reload(dynamic_constraints)
from kinematic_constraints import (
    AddInitialConstraints,
    AddLandingConstraints
)
from dynamic_constraints import (
    AddCollocationConstraints,
    EvaluateDynamics
)

In [768]:
# set up constants

N = 20     # number of collocation points
n_x = 9
n_u = 3

# where we start, where we go, mass?, 
# start position
r_0 = 3474.8*1000                 #radius of Mars, in km
alpha_0 = -23.45*np.pi/180   #longitude degrees
beta_0 = -2.94*np.pi/180   #latitude degrees
Vx_0 = 0
Vy_0 = 0
Vz_0 = 0
m_0 = 15234                  #kg
phi_0 = 90*np.pi/180         #pitch degrees
psi_0 = 0          #yaw degrees

# final position
r_N = 3474.8*1000                 #radius of Mars, in km
alpha_N = -23.45*np.pi/180   #degrees
beta_N = -1.94*np.pi/180     #degrees
Vx_N = 0
Vy_N = 0
Vz_N = 0
phi_N = 90*np.pi/180
psi_N = 0

# initial control input 
T_0 = 0
phidot_0 = 0
psidot_0 = 0

# final control input
T_N = 0
phidot_N = 0
psidot_N = 0

# constraint coefficients
r_min = 3474.8*1000
h_max = 30*1000
r_max = 3474.8*1000 + h_max     #h_max km above Mars surface
phi_min = -90*np.pi/180    #degree
phi_max = 90*np.pi/180     #degree
psi_min = -180*np.pi/180   #degree
psi_max = 180*np.pi/180    #degree
phidot_min = -8*np.pi/180  #deg/sec
phidot_max = 8*np.pi/180   #deg/sec
psidot_min = -8*np.pi/180  #deg/sec
psidot_max = 8*np.pi/180   #deg/sec
T_min = 0                  #Newton
T_max = 43148              #Newton

initial_state = np.array([r_0,alpha_0,beta_0,Vx_0,Vy_0,Vz_0,m_0,phi_0,psi_0])
final_state = np.array([r_N,alpha_N,beta_N,Vx_N,Vy_N,Vz_N,phi_N,psi_N])
input_limit = np.array([[T_min,phidot_min,psidot_min],[T_max,phidot_max,psidot_max]])
initial_input = np.array([T_0,phidot_0,psidot_0])
final_input = np.array([T_N,phidot_N,psidot_N])


# m = 8321.09        #kg
g = 3.711          #m/s^2
Isp = 302.39       #s (effective specific impulse of the lander, used in mdot equation)


In [769]:
# Initialize Mathematical Program and Add Variables

prog = MathematicalProgram()
x = np.zeros((N,n_x),dtype="object")
u = np.zeros((N,n_u),dtype="object")
dt = np.zeros((1,), dtype="object")

for i in range(N):
    x[i] = prog.NewContinuousVariables(n_x,"x_" + str(i))
    u[i] = prog.NewContinuousVariables(n_u,"u_" + str(i))
dt = prog.NewContinuousVariables(1,"dt") 

t0 = 0.0

In [770]:
# Add linear cost
gz = -x[N-1,6]
prog.AddLinearCost(gz)

In [771]:
 # Add dynamic constraint
AddCollocationConstraints(prog, N, x, u, dt)

In [772]:
# TODO: Add kinematic initial and final(landing) constraint
AddInitialConstraints(prog, N, x, u, initial_state, initial_input)
AddLandingConstraints(prog, N, x, u, final_state, final_input)

In [773]:
# TODO: Add input/state limitation as bounding box constraints: NEED CHECK

# height limit: 0 <= h <= hmax
prog.AddBoundingBoxConstraint(r_min, r_max, x[:,0])

# mass upper limit
# prog.AddBoundingBoxConstraint(0.1, m_0, x[:,6])
for i in range(N-1):
  prog.AddLinearConstraint(x[i+1, 6] <= x[i, 6])

# angular position input limit
prog.AddBoundingBoxConstraint(phi_min, phi_max, x[:,7]) #infeasible constraint
prog.AddBoundingBoxConstraint(psi_min, psi_max, x[:,8])

# thrust input limit
# prog.AddBoundingBoxConstraint(T_min, T_max, u[:,0])# change constraint

# omega angular velocity state limits
prog.AddBoundingBoxConstraint(phidot_min, phidot_max, u[:,1])
prog.AddBoundingBoxConstraint(psidot_min, psidot_max, u[:,2])

In [790]:
# Set up solver initial guess

# state initial guess
xinit = np.zeros((N,n_x))

# altitude initial guess
a1 = np.linspace(r_0,r_0+6,N-N//2)
a2 = np.linspace(r_0+5,r_0,N//2)
xinit[:,0] = np.concatenate((a1,a2),axis=0)

# longitude initial guess
a1 = np.linspace(alpha_0,-23.47*np.pi/180,N-N//2)
a2 = np.linspace(-23.46*np.pi/180,alpha_N,N//2)
xinit[:,1] = np.concatenate((a1,a2),axis=0)

# latitude initial guess
xinit[:,2] = np.linspace(beta_0,beta_N,N)

# velocity Vx
a1 = np.linspace(Vx_0,100,N//5)
a2 = np.linspace(100,-100,N-2*N//5)
a3 = np.linspace(-100,Vx_N,N//5)
xinit[:,3] = np.concatenate((a1,a2,a3),axis=0)

# velocity Vy no need to do anything, all zeros is a good guess

# velocity Vz
a1 = np.linspace(Vz_0,150,N-N//2)
a2 = np.linspace(140,Vz_N,N//2)
xinit[:,5] = np.concatenate((a1,a2),axis=0)
# mass
xinit[:,6] = np.full(N,m_0)

# pitch phi Need Halp
xinit[:,7] = np.full(N,phi_0)

# yaw psi
xinit[:,8] = np.full(N,psi_0)

# input initial guess
uinit = np.zeros((N,n_u))
for i in range(n_u):
    uinit[:,i] = np.random.uniform(input_limit[0,i],input_limit[1,i],N)

tinit = 0.5

prog.SetInitialGuess(u,uinit)
prog.SetInitialGuess(x,xinit)
prog.SetInitialGuess(dt, [tinit])

In [791]:
# set up solver
solver = SnoptSolver()
prog.SetSolverOption(SolverType.kSnopt, "Iterations limit", 1000000)
result = solver.Solve(prog)
print(result.get_solution_result())
# get solution of states and inputs from solver result
x_sol = result.GetSolution(x)
u_sol = result.GetSolution(u)
t_sol = result.GetSolution(dt)
timesteps = np.linspace(t0, t_sol*N, N)
xdot_sol = np.zeros(x_sol.shape)
for i in range(N):
    xdot_sol[i] = EvaluateDynamics(x_sol[i],u_sol[i])

# create trajectory by interpolating between collocation points using piecewise polynomial
x_traj = PiecewisePolynomial.CubicHermite(timesteps,x_sol.T,xdot_sol.T)
u_traj = PiecewisePolynomial.FirstOrderHold(timesteps,u_sol.T)

SolutionResult.kSolutionFound


In [792]:
print("Total mass lost:", xinit[N-1, 6] - x_sol[N-1, 6], "kg")

Total mass lost: 10307.188582771689 kg


In [ ]:
plt.plot(timesteps, x_sol[:,0], label="T")
plt.legend()

In [ ]:
plt.plot(timesteps, x_sol[:,1]*180/np.pi, label="phi")
# plt.plot(timesteps, x_sol[:,8]*180/np.pi, label="psi")
plt.legend()

In [ ]:
plt.plot(timesteps, x_sol[:,0]-3474.8*1000, label='r')
# plt.plot(timesteps, u_sol[:,0], label="T")
plt.xlabel("time (s)")
plt.legend()

In [ ]:
# TODO: create LQR to follow the trajectory

In [803]:
# TODO: simulate and animate path, create disturbance in simulation
